In [1]:
import os
from utils import *
from conv_util import *
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [32]:
# Misc
LOG_DIR = './log/'

# Training Parameters
learning_rate = 0.0001
num_epochs = 100
num_steps = 1000
batch_size = 256
display_step = 50

# Network Parameters
num_classes = 6
keep_rate = 1

repeat_size = 81

In [58]:
raw = pd.read_csv('./storks_obs_train.csv', header=None)

In [59]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))#.reshape((9, 9, 3)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).reshape((-1, 9, 9, 3))
Y = np.array(Y)
label_names = np.unique(Y)


le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [68]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [69]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    
    # Dataset Preparation
    trn_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(128).repeat().batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).shuffle(128).repeat().batch(batch_size)
    
    iterator = tf.data.Iterator.from_structure(trn_ds.output_types, trn_ds.output_shapes)
    x, y = iterator.get_next()
    
    training_init = iterator.make_initializer(trn_ds)
    testing_init = iterator.make_initializer(test_ds)
    
    # Logits and Predictions
    logits = build_model(x, keep_prob, input_dim=[9, 9], mean=mean) 
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        tf.summary.scalar('cross_entropy_loss', loss_op)
    
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], y)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Global Initializer
    initializer = tf.global_variables_initializer()
    
    # Summary
    summary = tf.summary.merge_all()
    
    # Global saver
    saver = tf.train.Saver()

In [70]:
MODEL_ID = 'storkes_plain'

In [71]:
with tf.Session(graph=model_graph) as sess:
    
    # Run Global Initializer
    sess.run(initializer)
    
    sess.run(training_init)
    for step in range(1, num_steps+1):
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={keep_prob: keep_rate})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    sess.run(testing_init)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={keep_prob: 1.0}))
    
    # Save model
    saver.save(sess, os.path.join(LOG_DIR, 'model_%s'%MODEL_ID, "final.model.ckpt"), step)

Step 1, Minibatch Loss= 1.9202, Training Accuracy= 0.477
Step 50, Minibatch Loss= 0.5225, Training Accuracy= 0.848
Step 100, Minibatch Loss= 0.3593, Training Accuracy= 0.906
Step 150, Minibatch Loss= 0.3073, Training Accuracy= 0.906
Step 200, Minibatch Loss= 0.2676, Training Accuracy= 0.910
Step 250, Minibatch Loss= 0.2282, Training Accuracy= 0.934
Step 300, Minibatch Loss= 0.2205, Training Accuracy= 0.934
Step 350, Minibatch Loss= 0.2237, Training Accuracy= 0.934
Step 400, Minibatch Loss= 0.1427, Training Accuracy= 0.969
Step 450, Minibatch Loss= 0.1884, Training Accuracy= 0.957
Step 500, Minibatch Loss= 0.1614, Training Accuracy= 0.953
Step 550, Minibatch Loss= 0.1370, Training Accuracy= 0.961
Step 600, Minibatch Loss= 0.1671, Training Accuracy= 0.961
Step 650, Minibatch Loss= 0.1184, Training Accuracy= 0.969
Step 700, Minibatch Loss= 0.0922, Training Accuracy= 0.977
Step 750, Minibatch Loss= 0.1404, Training Accuracy= 0.957
Step 800, Minibatch Loss= 0.0682, Training Accuracy= 0.988


In [53]:
with tf.Session(graph=model_graph) as sess:
    
    # Restore pretrained model
    saver.restore(sess, './log/model_storkes_plain/final.model.ckpt-1000')
    print('Model restored.')
    
    sess.run(testing_init)
    print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={keep_prob: 1.0}))
    

INFO:tensorflow:Restoring parameters from ./log/model_storkes_plain/final.model.ckpt-1000
Model restored.
Testing Accuracy: 0.94140625
